In [2]:
# --------------------------------------------------------------------------

# ----------------- design to establish all conditions for  unsupervised sample selection

# -------------------------------------------------------------------------


# ¡¡¡ --- !!! # ---> modules and data cases

# --- system modules

import sys
import datetime
import os


base_dir = "/home/u0106869/vfonsecad/kul_phd/programming/phd_valeria_fonseca_diaz_wp1/wp1_study002_sample_selection"

# --- data handling modules

import numpy as np
import pandas as pd
import scipy.io as sp_io
import scipy as sp

# --- visualization modules

import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import rcParams
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- my modules

methods_dir = base_dir + '/methodology/python'  
sys.path.insert(0, methods_dir + '/model_building')
sys.path.insert(0, methods_dir + '/read_data')
sys.path.insert(0, methods_dir + '/sample_selection')
from class_chemometrics_data import chemometrics_data
from class_mcw_pls import mcw_pls, mcw_pls_sklearn, min_rmsecv_lv_simpls
from class_sample_selection import sample_selection, covmap
import simpls_module



# ¡¡¡ --- !!! # ---> base working directory and available data cases


# ************************************ init --- user 
cases_dict = {"d0023": ["d0023_milkrobot_nirsensor2017", "d0023_data_prepared_06"],
              "d0024": ["d0024_pig_manure", "d0024_data_prepared_04"],             
             "d0025": ["d0025_apples_els", "d0025_data_prepared_04"]}
# ************************************ end --- user 


print("--------- imports loaded ----------")




# ¡¡¡ --- !!! # ---> data


# ************************************ init --- user 
caseID_key = "d0023"
expeID = "2020-09-29_0001"
# ************************************ end --- user 

case_dir = cases_dict[caseID_key][0]
dname = cases_dict[caseID_key][1]
data_dir = '/data/' + case_dir + '/data_prepared/'





# ************************************ init --- user
data_class = chemometrics_data(base_dir + data_dir + dname + '.mat', 
                               data_identifier = data_dir + dname,
                               include_val = False,
                               include_test = False,
                               include_unlabeled = False,
                               y_all_range = False,
                               y_range = np.array([0]),
                               obs_all_cal = True,
                              shuffle = False)
# ************************************ end --- user



print(data_class.ncal, data_class.K)
print("--------- data loaded for " + data_class.data_identifier + "----------")






--------- imports loaded ----------
316 256
--------- data loaded for /data/d0023_milkrobot_nirsensor2017/data_prepared/d0023_data_prepared_06!*fat    *!----------


In [3]:
# --- factors


npc_range = list(range(1,31))
npc_range.append(data_class.K)
method_name = ["random","ks","duplex","puchwein","clustering","optfederov"]
sample_sizes = list(range(30,data_class.ncal,10))



In [17]:
# --- making design


design = []


        
for pc in npc_range:

    for method in method_name:

        for ss in sample_sizes:
            
            if method!="optfederov" or (pc<=30 and (ss-pc)>=5):                

                run = {"npc":pc,
                       "method_name":method,
                       "sample_size":ss}
                
            

                design.append(run)
                    
# pd.DataFrame(design)
print("done")
design[-1]
len(design)

done


5360

In [23]:
df = pd.DataFrame(design)
df[(df["method_name"]=="optfederov")&(df["sample_size"]<40)]

,method_name,npc,sample_size
145,optfederov,1,30
319,optfederov,2,30
493,optfederov,3,30
667,optfederov,4,30
841,optfederov,5,30
1015,optfederov,6,30
1189,optfederov,7,30
1363,optfederov,8,30
1537,optfederov,9,30
1711,optfederov,10,30


In [24]:
# --- all iterations for sample selection

from datetime import datetime

print('finished: ',datetime.now())



selected_samples_dict = {}


for ii in range(len(design)):
    
    if ii%100==0:
        print(ii)

    

    design_setting = design[ii]    

    # - class sample selection

    my_sample_selection = sample_selection(data_class.get_cal()["xcal"], ncp = design_setting["npc"])  
    my_sample_selection.get_xcal_pca_scores(first_ncp = 0)

    # - sample size

    n_sel = design_setting["sample_size"]

    # - input matrix type for sample selection function arguments

    if design_setting["npc"]>30:

        dim_red = False
        dist_measure = "euclidean"

    else:

        dim_red = True
        dist_measure = "mahalanobis"

    # - select samples based on all settings depending on method

    if design_setting["method_name"] == "random":

        current_samples = my_sample_selection.random_sample(Nout = n_sel)["sample_id"]

    if design_setting["method_name"] == "ks":

        current_samples = my_sample_selection.kennard_stone(Nout = n_sel, fixed_samples=None, dim_reduction=dim_red, distance_measure=dist_measure)["sample_id"]

    if design_setting["method_name"] == "duplex":

        current_samples = my_sample_selection.duplex(Nout = n_sel, dim_reduction=dim_red, distance_measure=dist_measure)["sample_id"]

    if design_setting["method_name"] == "puchwein":

        current_samples = my_sample_selection.puchwein(Nout = n_sel, factor_k=0.0001, dim_reduction=dim_red, distance_measure=dist_measure)["sample_id"]

    if design_setting["method_name"] == "clustering":

        current_samples = my_sample_selection.clustering(Nout = n_sel , dim_reduction=dim_red, distance_measure=dist_measure)["sample_id"]

    if design_setting["method_name"] == "optfederov" and design_setting["npc"] <= 30:

        current_samples = my_sample_selection.optfederov_r(Nout = n_sel, fixed_samples=None, optimality_criterion='D')["sample_id"]


    # - save result

    design_setting["selected_samples"] = current_samples.flatten()


    selected_samples_dict[str(ii)] = design_setting


print('finished: ',datetime.now())

# ¡¡¡ --- !!! ---> save output 

sp_io.savemat(base_dir + "/experiments/python/" + expeID + "/output/" + caseID_key + "_01_design_selected_samples.mat", selected_samples_dict)
df_output = pd.DataFrame.from_dict(selected_samples_dict, orient="index")
df_output.to_pickle(base_dir + "/experiments/python/" + expeID + "/output/" + caseID_key + "_01_design_selected_samples.pkl")

finished:  2020-09-24 11:54:52.826249
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
finished:  2020-09-24 12:04:42.056994
